- 불완전한 번역 추론 결과임
    - 포르투갈어가 영어로 번역되지 않은 경우가 있음
    
    - 영어가 한글로 번역되지 않은 경우가 있음

- 결과물 설명
    - `../artifact/reviews_with_senti/eng_title_with_senti.csv`
        - 영어 리뷰 제목과 감성분석 추론 결과

    - `../artifact/reviews_with_senti/eng_message_with_senti.csv`
        - 영어 리뷰 내용과 감성분석 추론 결과




In [11]:
import pandas as pd
import os

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [12]:
TRANSLATED_RESULT_DIR = r"../downloads/backup"
ARTIFACT_DIR = r"../artifact/reviews_with_senti/"

In [ ]:
preprocessed_texts_path = os.path.join(TRANSLATED_RESULT_DIR, 'all_portuguess_final.txt')             # 전처리를 통해 중복을 제거한 포르투갈어. 문장의 길이로 정렬함
preprocessed_with_id_path = os.path.join(TRANSLATED_RESULT_DIR, 'reviews_final.csv')     # 원본을 전처리된 포르투갈어로 대체한 데이터. id 존재
p2e_path = os.path.join(TRANSLATED_RESULT_DIR, 'trans_p2e_final.txt')               # 포르투갈어 -> 영어
e2k_path = os.path.join(TRANSLATED_RESULT_DIR, 'trans_e2k_final.txt')               # 영어 -> 한국어

for path in [preprocessed_with_id_path, preprocessed_texts_path, p2e_path, e2k_path]:
    print(os.path.exists(path))

True
True
True
True


In [14]:
translated_result_df = []
for path in [preprocessed_texts_path, p2e_path, e2k_path]:
    df = pd.read_csv(
        path,
        header=None,
        names=["review"],
        sep=r'\n',
        engine='python',
        quotechar='"',
        doublequote=True
    )
    print(df.shape)
    translated_result_df.append(df)

merged_translation = pd.concat(translated_result_df, axis=1)
merged_translation.columns = ['preprocessed', 'p2e', 'e2k']

# 후처리 필요...
for col in ["preprocessed", "p2e", "e2k"]:
    # 1. 이중 따옴표 먼저 처리
    merged_translation[col] = merged_translation[col].str.replace('""', '"', regex=False)
    # 2. 시작과 끝의 따옴표만 제거 (정규표현식 사용)
    merged_translation[col] = merged_translation[col].str.replace(r'^"(.*)"$', r'\1', regex=True)

merged_translation.shape

(38091, 1)
(38091, 1)
(38091, 1)


(38091, 3)

In [15]:
sentimental_score_path = os.path.join(TRANSLATED_RESULT_DIR, 'senti_eng_final.csv')   # 영어 문장에 대한 감성분석 추론 결과
sentimental_score_df = pd.read_csv(sentimental_score_path)
sentimental_score_df.shape

(38091, 3)

In [16]:
sentimental_score_df['max_sentimental'] = sentimental_score_df.idxmax(axis=1)

In [17]:
inference_result = pd.concat([merged_translation, sentimental_score_df], axis=1)
inference_result.drop_duplicates(inplace=True)
inference_result.shape

(37500, 7)

In [18]:
inference_result[inference_result.p2e.isnull()]

,preprocessed,p2e,e2k,negative,neutral,positive,max_sentimental
37980,n/d,NaN,NaN,0.002729,0.99661,0.000661,neutral


In [19]:
# 동일한 영어 문장의 감성 분석 결과가 다른 경우를 있었음을 확인했으므로, 이에 대한 처리 진행

# 1. p2e별로 max_sentimental 값이 몇 종류 있는지 계산
label_conflicts = (
    inference_result
    .groupby('p2e')['max_sentimental']
    .nunique()
    .gt(1)  # 그룹 내 감성 레이블이 2개 이상이면 True
)

# 2. 레이블이 다른 p2e 목록 추출
conflicting_p2e = label_conflicts[label_conflicts].index

# 3. 해당 p2e를 가진 모든 행 추출
conflict_rows = inference_result[inference_result['p2e'].isin(conflicting_p2e)]
conflict_rows

,preprocessed,p2e,e2k,negative,neutral,positive,max_sentimental
35143,recomendaria sim,I would recommend it,추천합니다.,0.001551,0.040197,0.958252,positive
36155,"recomendo, sim",I would recommend it,추천할 것입니다.,0.001097,0.843380,0.155523,neutral


In [20]:
inference_result.loc[[35143, 36155], ['negative','neutral','positive','max_sentimental']] = \
    inference_result.loc[35143, ['negative','neutral','positive','max_sentimental']].values

In [21]:
# 변환 확인
inference_result[inference_result['p2e'].isin(conflicting_p2e)]

,preprocessed,p2e,e2k,negative,neutral,positive,max_sentimental
35143,recomendaria sim,I would recommend it,추천합니다.,0.001551,0.040197,0.958252,positive
36155,"recomendo, sim",I would recommend it,추천할 것입니다.,0.001551,0.040197,0.958252,positive


In [22]:
inference_result.drop_duplicates(inplace=True)
inference_result.shape

(37500, 7)

In [23]:
inference_result.drop_duplicates(inplace=True, subset='preprocessed')
inference_result.shape

(37454, 7)

In [24]:
inference_result[inference_result['preprocessed'].duplicated(keep=False)].sort_values('p2e')

,preprocessed,p2e,e2k,negative,neutral,positive,max_sentimental


In [25]:
# 최종 파일 생성
preprocessed_with_id_reviews_df = pd.read_csv(preprocessed_with_id_path)
preprocessed_with_id_reviews_df.shape

(42590, 3)

In [26]:
preprocessed_with_id_comment_title = preprocessed_with_id_reviews_df[['review_id', 'review_comment_title']].dropna()
preprocessed_with_id_commnet_message = preprocessed_with_id_reviews_df[['review_id', 'review_comment_message']].dropna()

In [27]:
eng_comment_title_df = pd.merge(preprocessed_with_id_comment_title, inference_result, left_on="review_comment_title", right_on='preprocessed', how='left')
eng_comment_message_df = pd.merge(preprocessed_with_id_commnet_message, inference_result, left_on="review_comment_message", right_on='preprocessed', how='left')

In [28]:
eng_comment_title_df = eng_comment_title_df[['review_id', 'negative','neutral','positive','max_sentimental', 'preprocessed','p2e','e2k']]
eng_comment_message_df = eng_comment_message_df[['review_id', 'negative','neutral','positive','max_sentimental', 'preprocessed','p2e','e2k']]

save_path = os.path.join(ARTIFACT_DIR, 'eng_kor_title_with_senti.csv')
eng_comment_title_df.drop_duplicates(inplace=True)
eng_comment_title_df.to_csv(save_path, index=False)
print(save_path)

save_path = os.path.join(ARTIFACT_DIR, 'eng_kor_message_with_senti.csv')
eng_comment_message_df.drop_duplicates(inplace=True)
eng_comment_message_df.to_csv(save_path, index=False)
print(save_path)

../artifact/reviews_with_senti/eng_kor_title_with_senti.csv
../artifact/reviews_with_senti/eng_kor_message_with_senti.csv


### 번역이 이상한 부분을 발견하여 데이터 확인

In [29]:
# import langid
# import re

# def detect_language(text, 
#                     length_threshold=20, 
#                     alpha_threshold=0.7, 
#                     score_threshold=0.85):
#     """
#     텍스트가 영어인지 판별.
#     - 짧은 문장은 알파벳 비율 기반으로 판별
#     - 긴 문장은 langid + 보정 로직
#     """
#     if not isinstance(text, str) or not text.strip():
#         return 'unknown'

#     text = text.strip()
#     text_len = len(text)
    
#     # 1. 짧은 문장: 알파벳 비율 기준
#     if text_len < length_threshold:
#         letters = re.findall(r"[A-Za-z]", text)
#         ratio = len(letters) / text_len
#         if ratio >= alpha_threshold:
#             return 'en-lite'  # 영어 가능성 높음
#         else:
#             return 'non-en-lite'  # 영어 아님으로 간주

#     # 2. 긴 문장: langid 사용
#     try:
#         lang, score = langid.classify(text)
#         # 오탐 보정: 영어 알파벳으로만 되어 있는데 et/pt 나오면 영어로 간주
#         if lang in ['et', 'pt']:
#             if re.fullmatch(r"[A-Za-z\s.,!?'\"]+", text):
#                 return 'en-fallback'
#         if lang == 'en' and score >= score_threshold:
#             return 'en'
#         return lang
#     except:
#         return 'unknown'

# eng_comment_title_df['lang_result'] = eng_comment_title_df['p2e'].apply(detect_language)
# eng_comment_message_df['lang_result'] = eng_comment_message_df['p2e'].apply(detect_language)

In [30]:
import re

# 숫자만인지 판별하는 함수
def is_numeric_only(text):
    if not isinstance(text, str):
        return False
    return bool(re.fullmatch(r"[0-9\s.,%-]+", text))

# 숫자/영어 둘 다 제외
filtered_title_df = eng_comment_title_df[
    (~eng_comment_title_df['lang_result'].isin(['en', 'en-lite', 'en-fallback'])) &  # 영어 제외
    (~eng_comment_title_df['p2e'].apply(is_numeric_only))                            # 숫자 제외
]
filtered_message_df = eng_comment_message_df[
    (~eng_comment_message_df['lang_result'].isin(['en', 'en-lite', 'en-fallback'])) &  # 영어 제외
    (~eng_comment_message_df['p2e'].apply(is_numeric_only))                            # 숫자 제외
]


KeyError: 'lang_result'

#### gpt를 이용하여 수정
Issue links
- https://github.com/jmhwang-dev/e-commerce/issues/31
- https://github.com/jmhwang-dev/e-commerce/issues/32

In [ ]:
# filtered_title_df[['review_id', 'p2e']].to_csv('check_title.csv')
# filtered_message_df[['review_id', 'p2e']].to_csv('check_message.csv')